###导入相关的库

In [1]:
import requests
from lxml import etree
import re
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import urllib.request

###爬取上市公司的股票代码

In [2]:
res = requests.get('https://hq.gucheng.com/gpdmylb.html')
res_xpath = etree.HTML(res.text)
node = res_xpath.xpath('//*[@id="stock_index_right"]/div[3]/section/a[@href]/text()')
codeall = re.findall(r'\d{6}', str(node))
code_list = []
for node in codeall:
    code_list.append(node)
gp_list = code_list[3:1003]#股票代码
# gp_list

###爬取1000家公司2010年到2020年的股票交易数据

In [3]:
start_date = "20151118"#股票交易的起始日期
end_date = "20201118"#股票交易数据的终止日期
for gp in gp_list:
        download_url = "http://quotes.money.163.com/service/chddata.html?code=1"+gp+"&start="+start_date+"&end="+end_date+"&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP"        
        #以6和3开头的股票代码code前都有0，以0开头的股票代码code前都有1，由于我爬取的股票都是0开头的，所以不考虑前一种情况
        data = requests.get(download_url)
        f = open(gp+'.csv','wb')     
        for chunk in data.iter_content(chunk_size=10000):
            if chunk:
                f.write(chunk)#保存获取的股票交易数据

###爬取1000家公司的公告新闻

In [7]:
# for node in codeall:
#     code_list.append(node)
gp_list = code_list[616:1003]#获取对应公司的股票代码
for dm in gp_list:
    x=0
    a0,a1,a2,a3,a4=[],[],[],[],[]
    while True:
        url = 'http://quotes.money.163.com/f10/gsgg_'+dm+',zjgg,'+str(x)+'.html'
        x+=1
        mes = requests.get(url)
        selector = etree.HTML(mes.text)  
        data = re.compile(r'暂无数据')
        data1 = re.findall(data,mes.text)
        if data1 == ['暂无数据']:
            break
        mes = requests.get(url)
        selector = etree.HTML(mes.text)  
        data = re.compile(r'<td class="align_c">\d{4}-\d{2}-\d{2}</td>')#正则匹配日期
        data1 = re.findall(data,mes.text)
        data_list=[]
        for d in data1:
            data = re.compile(r'\d{4}-\d{2}-\d{2}')
            data0 = re.findall(data,d)
            data_list.append(data0)#获取公告内的日期
        selector_xpath1 = selector.xpath('//tr/td/a/@href')#提取网页中有关公告的链接
        selector_xpath2 = selector.xpath('//tr/td/a/@title')#提取网页中有关公告的文字
        a=selector_xpath1[10:]#前十个链接都与公告无关，故不提取
        b=selector_xpath2[9:] #获取公告名称
        for i,j ,k in zip(a,b,data_list):
            html='http://quotes.money.163.com/'+i#获取完整的公告链接
            response = requests.get(html)
            text = response.text
            compil = re.compile(r'[\u4e00-\u9fa5。，？！“”：（）—%.、0-9]')#正则匹配公告中的中文
            han = re.findall(compil,text)#正则匹配公告中的中文
            a0.append(k[0])
            a1.append(html)
            a2.append(j)
            var = " "
            for f in han:
                var+=f
            a3.append(var)
            a4.append(dm)
    result = pd.DataFrame(zip(a0,a4,a1,a2,a3))
    result.to_excel(f'{dm}公告.xlsx')#将公告内容保存在文件中